# Reinforecement Learning for Traffic Light Control

Required packages or environments:

* Python 3.6
* SUMO 0.32 with TraCI module. Click the [link](https://sumo.dlr.de/docs/SUMO_User_Documentation.html) for user documentation 
* Keras 2.2.0 and Tensorflow 1.9.0

## Setting Up the environment through SUMO and TRACI

In [1]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import argparse
import random
from pprint import pprint as pp
import numpy as np

### Import python modules from the SUMO_HOME/tools directory

In [2]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

### Simulation environment setting up

In [3]:
from sumolib import checkBinary  
import traci  # noqa
import traci.constants as tc

### Generating route file (data/road.rou.xml) based on the demand per second from differnt directions

In [4]:
def generate_routefile():
    random.seed(42)  # make tests reproducible
    N = 3600  # number of time steps
    # demand per second from different directions
    pEB = 1. / 10
    pWB = 1. / 11
    pSB = 1. / 15
    pNB = 1. / 25
    with open("data/road.rou.xml", "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" \
guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="7" minGap="3" maxSpeed="25" guiShape="bus"/>

        <route id="EastBound" edges="W-0W 0W-0 0-0E 0E-E" />
        <route id="WestBound" edges="E-0E 0E-0 0-0W 0W-W" />
        <route id="SouthBound" edges="N-0N 0N-0 0-0S 0S-S" />
        <route id="NorthBound" edges="S-0S 0S-0 0-0N 0N-N" />""", file=routes)
        vehNr = 0
        for i in range(N):
            if random.uniform(0, 1) < pEB:
                print('    <vehicle id="EastBound_%i" type="typeWE" route="EastBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pWB:
                print('    <vehicle id="WestBound_%i" type="typeWE" route="WestBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pSB:
                print('    <vehicle id="SouthBound_%i" type="typeNS" route="SouthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pNB:
                print('    <vehicle id="NorthBound_%i" type="typeNS" route="NorthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
        print("</routes>", file=routes)

In [5]:
# The program looks like this
#    <tlLogic id="tl0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="41" state="GGrrGGrr"/>
#        <phase duration="4" state="yyrryyrr"/>
#        <phase duration="41" state="rrGGrrGG"/>
#        <phase duration="4" state="rryyrryy"/>
#    </tlLogic>

In [6]:
def run():
    """execute the TraCI control loop"""
#    step = 0
    # we start with phase 2 where EW has green
   
    traci.junction.subscribeContext("tl0", tc.CMD_GET_VEHICLE_VARIABLE, 300, [tc.VAR_SPEED, tc.VAR_WAITING_TIME]) 
    print(traci.junction.getContextSubscriptionResults("tl0"))
#   traci.trafficlight.setPhase("tl0", 2)
#    while traci.lane.getLastStepVehicleNumber("0W-0_0") <200:
#   while traci.simulation.getMinExpectedNumber() > 0:        
    for step in range(10):
        print(f'\n_______________' \
              f"(step {step}):", end=' ')
        print(f"Delta = {traci.simulation.getDeltaT()} s", end='  _  ')
        print(f'# MinExpectedNumber: {traci.simulation.getMinExpectedNumber()}', end='  ')
        
        print(f'TL Phase {traci.trafficlight.getPhase("tl0")} _ ' \
              f'{traci.trafficlight.getRedYellowGreenState("tl0")}' \
              f'({traci.trafficlight.getPhaseDuration("tl0")} s) _ '\
              f'next phase at {traci.trafficlight.getNextSwitch("tl0")}s')
        
        traci.simulationStep()
        print(f' #EB approaching TL : {traci.edge.getLastStepVehicleNumber("0W-0")}' \
              f'  _  # vehicles waiting: {traci.edge.getLastStepHaltingNumber("0W-0")}' \
              f'EB ttl waiting time: {traci.edge.getWaitingTime("0W-0")}')
        print(traci.junction.getContextSubscriptionResults("tl0"))
#    if traci.trafficlight.getPhase("tl0") == 2:
            # we are not already switching
#           if traci.inductionloop.getLastStepVehicleNumber("tl0") > 0:
                # there is a vehicle from the north, switch
#       traci.trafficlight.setPhase("tl0", 3)
#           else:
                # otherwise try to keep green for EW
#               traci.trafficlight.setPhase("tl0", 2)
#        eb = traci.lane.getLastStepVehicleNumber("0W-0_0")
#        step += 1
#        print("Step "+ str(step))
#        print('Eastbound vechicles : ', eb)
    #traci.close()
    #sys.stdout.flush()


### TrafficSignal Class for traffic lights of an intersection,  via TRACI API to retriev status info and change traffice lights phases. 

In [7]:
class TrafficSignal:
    """
    This class represents a Traffic Lights of an intersection,
    via TRACI API to retriev status info and change traffice lights phases.
    """
    def __init__(self, tlsID,programID,delta_time, min_green, max_green, phases):
        self.id = tlsID
        self.programID = programID
        self.time_on_phase = 0.0
        self.delta_time = delta_time
        self.min_green = min_green
        self.max_green = max_green
        self.green_phase = 0
        self.num_green_phases = len(phases) // 2
        self.lanes = list(dict.fromkeys(traci.trafficlight.getControlledLanes(self.id)))  # remove duplicates and keep order
        self.edges = self._compute_edges()
        self.edges_capacity = self._compute_edges_capacity()

        logic = traci.trafficlight.Logic("new-program", 0, 0, phases=phases)
        traci.trafficlight.setCompleteRedYellowGreenDefinition(self.id, logic)


In [8]:
import argparse
def get_options():
    optParser = argparse.ArgumentParser()
    optParser.add_argument("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
#    options, args = optParser.parse_args()
    options, args = optParser.parse_known_args()
#    optParser.print_help()
    return options

this is the main entry point of the simulation

In [9]:
if __name__ == "__main__":
    options = get_options()
#    print(options)
    # this script has been called from the command line. It will start sumo as a
    # server, then connect and run
    if options.nogui:
        sumoBinary = checkBinary('sumo')
    else:
        sumoBinary = checkBinary('sumo-gui')

    # first, generate the route file for this simulation
    generate_routefile()

    # this is the normal way of using traci. sumo is started as a
    # subprocess and then the python script connects and runs
    traci.start([sumoBinary, "-c", "data/road.sumocfg",
                             "--tripinfo-output", "tripinfo.xml"])
    class durationRewardStepListener(traci.StepListener):
        def step(self, t=0):

            print("ExampleListener called at time %s ms." % traci.simulation.getTime())

            return True

    duration_listener = durationRewardStepListener()
    traci.addStepListener(duration_listener)
        
    run()

 Retrying in 1 seconds
Could not connect to TraCI server at localhost:50452 [Errno 61] Connection refused
 Retrying in 2 seconds
None

_______________(step 0): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phase at 41.0s
ExampleListener called at time 1.0 ms.
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 1): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phase at 41.0s
ExampleListener called at time 2.0 ms.
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 2): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phase at 41.0s
ExampleListener called at time 3.0 ms.
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 3): Delta = 1.0 s  _  # MinExpectedNumber: 54  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phase at 41.0s
ExampleListener called at time 

In [10]:
junc_id = traci.junction.getIDList()

In [11]:
sumoBinary


'/usr/local/opt/sumo/share/sumo/bin/sumo-gui'

In [12]:
!ps ax | grep sumo-gui

65892   ??  S      0:00.95 /usr/local/opt/sumo/share/sumo/bin/sumo-gui -c data/road.sumocfg --tripinfo-output tripinfo.xml --remote-port 50452
66137 s006  Ss+    0:00.02 /bin/sh -c ps ax | grep sumo-gui
66139 s006  S+     0:00.01 grep sumo-gui


In [13]:
class Env_TLC:
    """
    Class for Traffic Light Control Environment which is used for retrieving State,
    changing the traffic phases or duration via TRACI API

    """

    def __init__(self, programID, tlsID):
        self.ID = tlsID
        self.programID = programID
        self.lanes = traci.trafficlight.getControlledLanes(self.ID)
        self.links = traci.trafficlight.getControlledLinks(lgt_1)
        self.RYG_definition = traci.trafficlight.getCompleteRedYellowGreenDefinition(lgt_1)
        self.last_state = {}
        
        self.IBlaneList = [self.links[i][0][0]for i in range(len(self.links))]
        self.OBlaneList = [self.links[i][0][1]for i in range(len(self.links))]
        self.linkList = [self.links[i][0][2]for i in range(len(self.links))]
    
    def getStateArray(state):
        return np.row_stack([np.array(each) for each in tl0.last_state.values()])

    def updateLastState(self, ):

        self.last_state['IBOccupancy'] = [traci.lane.getLastStepOccupancy(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBVolume'] = [traci.lane.getLastStepVehicleNumber(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBMeanSpeed'] = [traci.lane.getLastStepMeanSpeed(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBQueuSize'] = [traci.lane.getLastStepHaltingNumber(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBWaitingTime'] = [traci.lane.getWaitingTime(
            laneID) for laneID in self.IBlaneList]

        self.last_state['OBOccupancy'] = [traci.lane.getLastStepOccupancy(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBVolume'] = [traci.lane.getLastStepVehicleNumber(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBMeanSpeed'] = [traci.lane.getLastStepMeanSpeed(
            laneID) for laneID in self.IBlaneList]
        self.last_state['OBQueuSize'] = [traci.lane.getLastStepHaltingNumber(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBWaitingTime'] = [traci.lane.getWaitingTime(
            laneID) for laneID in self.OBlaneList]

In [ ]:
tfcLgts =traci.trafficlight.getIDList()
lgt_1 = tfcLgts[0]
light1_Logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(lgt_1)[0]

In [ ]:
tl0 = Env_TLC(programID=light1_Logic.programID, tlsID=lgt_1)

In [ ]:
tl0.updateLastState()
state = tl0.getStateArray()

In [ ]:
tl0.last_state.keys()

In [ ]:
light1_Logic

In [ ]:
tl0.links[0][0]

In [ ]:
from traci import trafficlight as TL
from traci import lane as Ln
from traci import simulation as Sim

In [ ]:
TL.getNextSwitch(lgt_1)

In [ ]:
TL.setParameter(lgt_1, param='programID', value='testSetParam')

In [ ]:
TL.getProgram('tl0')

In [ ]:
Sim.saveState('test_save_state.xml')

In [ ]:
TL.getPhase('tl0')

In [ ]:
light1_Logic

In [ ]:
light1_Logic.currentPhaseIndex

In [ ]:
tc.CMD_GET_VEHICLE_VARIABLE

In [ ]:
traci.junction.subscribeContext("tl0", tc.CMD_GET_VEHICLE_VARIABLE, 300, [tc.VAR_SPEED, tc.VAR_WAITING_TIME]) 

In [ ]:
light1_Logic.phases[0]

In [ ]:
light1_Logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(lgt_1)[0]

In [ ]:
light1_Logic

In [ ]:
TL.setCompleteRedYellowGreenDefinition(lgt_1, light1_Logic)

In [ ]:
phases = light1_Logic.getPhases()

In [ ]:
phases

In [ ]:
phases[(TL.getPhase(lgt_1) + 2) % 4].duration = 50

In [ ]:
TL.setCompleteRedYellowGreenDefinition(lgt_1, light1_Logic)

In [ ]:
TL.getCompleteRedYellowGreenDefinition(lgt_1)

In [ ]:
current_duration = TL.getCompleteRedYellowGreenDefinition(lgt_1)[0].getPhases()[(TL.getPhase(lgt_1)) % 4].duration
next_duration = TL.getCompleteRedYellowGreenDefinition(lgt_1)[0].getPhases()[(TL.getPhase(lgt_1) + 2) % 4].duration
print(current_duration)
next_duration

In [ ]:
TL.getNextSwitch(lgt_1)

In [ ]:
traci.simulation.getTime()

In [ ]:
TL.getPhase(lgt_1)

In [ ]:
print(f'at time {traci.simulation.getTime()}     Phase before: {TL.getPhase(lgt_1)}')
traci.simulationStep()
print(f'at time {traci.simulation.getTime()}     Phase after: {TL.getPhase(lgt_1)}')

In [ ]:
class testListen(traci.StepListener):
    def step(self, t=0):

        print(f'at time {traci.simulation.getTime()}      During:   {TL.getPhase(lgt_1)}')

        return True

test_listen = testListen()
traci.addStepListener(test_listen)

In [ ]:
def Stop():
    try:
        traci.close(False)
    except Exception as e:
        pass
    return

In [ ]:
def Restart():
    traci.start([sumoBinary, '-c', 'data/road.sumocfg', '--load-state', 'test_save_state.xml', '--output-prefix', 'TIME']) 
    return

In [49]:
run()

{'EastBound_122': {64: 3.02248143637724, 122: 0.0}, 'EastBound_125': {64: 14.348034219164408, 122: 0.0}, 'EastBound_126': {64: 9.826344391517344, 122: 0.0}, 'EastBound_127': {64: 6.897016803175211, 122: 0.0}, 'EastBound_135': {64: 2.428692129254341, 122: 0.0}, 'NorthBound_130': {64: 9.26513050077483, 122: 0.0}, 'SouthBound_120': {64: 15.524716057825342, 122: 0.0}, 'SouthBound_123': {64: 4.7567908845308, 122: 0.0}, 'SouthBound_128': {64: 10.732421405427162, 122: 0.0}, 'SouthBound_132': {64: 5.999725664593279, 122: 0.0}, 'SouthBound_133': {64: 1.86436680406332, 122: 0.0}, 'WestBound_117': {64: 3.504378092475236, 122: 0.0}, 'WestBound_124': {64: 16.619253268446776, 122: 0.0}, 'WestBound_129': {64: 9.603915431164204, 122: 0.0}, 'WestBound_131': {64: 5.907877949718385, 122: 0.0}, 'WestBound_134': {64: 3.09933735365209, 122: 0.0}}

_______________(step 0): Delta = 1.0 s  _  # MinExpectedNumber: 50  TL Phase 2 _ rrGGrrGG(41.0 s) _ next phase at 536.0s
ExampleListener called at time 502.0 ms.


In [50]:
TL.getCompleteRedYellowGreenDefinition(lgt_1)[0].getPhases()[TL.getPhase(lgt_1)].duration

41.0

In [51]:
TL.getPhaseDuration(lgt_1)

41.0

In [52]:
light1_Logic.getPhases()[(TL.getPhase(lgt_1) + 2) % 4].duration

50

In [54]:
laneareas = traci.lanearea.getIDList()

In [55]:
traci.constants.RESPONSE_SUBSCRIBE_LANEAREA_VARIABLE

237

In [61]:
traci.lanearea.getContextSubscriptionResults(laneareas[0])

In [62]:
for i in range(50):
    value = traci.lanearea.getAllContextSubscriptionResults() 
    if len(value) is not 0  :  print(value)
    traci.simulation.step()

ExampleListener called at time 1002.0 ms.
at time 1002.0      During:   2
ExampleListener called at time 1003.0 ms.
at time 1003.0      During:   2
ExampleListener called at time 1004.0 ms.
at time 1004.0      During:   2
ExampleListener called at time 1005.0 ms.
at time 1005.0      During:   2
ExampleListener called at time 1006.0 ms.
at time 1006.0      During:   2
ExampleListener called at time 1007.0 ms.
at time 1007.0      During:   2
ExampleListener called at time 1008.0 ms.
at time 1008.0      During:   2
ExampleListener called at time 1009.0 ms.
at time 1009.0      During:   2
ExampleListener called at time 1010.0 ms.
at time 1010.0      During:   2
ExampleListener called at time 1011.0 ms.
at time 1011.0      During:   2
ExampleListener called at time 1012.0 ms.
at time 1012.0      During:   2
ExampleListener called at time 1013.0 ms.
at time 1013.0      During:   2
ExampleListener called at time 1014.0 ms.
at time 1014.0      During:   2
ExampleListener called at time 1015.0 

In [ ]:
StopnRestart()

In [ ]:
import sumolib

In [ ]:
output = sumolib.output.parse('data/e2output.xml', element_names=['interval'])

In [ ]:
detector_output = []
for each in output:
    detector_output.append(each)

In [ ]:
detector_output = [each for each in output]

In [ ]:
detector_output[0].getAttributes()

In [ ]:
from traci import lanearea as La

In [ ]:
OBDetectors = La.getIDList()

In [ ]:
Total_VehIds=[]
Current_VehIDs = []
for _ in range(800):
    for each in [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in OBDetectors]:
        for ID in each:
            if ID not in Total_VehIds:
                Total_VehIds.append(ID)
    #LastStep_VehIDs = [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in laneareas]
    traci.simulationStep()

In [ ]:
Stop()
Restart()

In [ ]:
len(TotalIds)

In [ ]:
for each in [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in OBDetectors]:
        for ID in each:
             Current_vehIDs.append(ID)

In [ ]:
Current_vehIDs

In [ ]:
def OB_nbr_VehLeft(seconds, OBDetectors):
    Starting_VehIDs = []
    Total_VehIDs = []
    Current_VehIDs = []
    
    # Query for Starting Vehicles in OBDetector Areas
    for each in [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in OBDetectors]:
            for ID in each:
                 Starting_VehIDs.append(ID)
                    
    # Query for Total Vehicles in OBDetector Areas
    for _ in range(seconds):
        for each in [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in OBDetectors]:
            for ID in each:
                if ID not in Total_VehIDs:
                    Total_VehIDs.append(ID)
        traci.simulationStep()
        
    # Query for remaining Vehicles in OBDetector Areas
    for each in [list(La.getLastStepVehicleIDs(lane_area)) for lane_area in OBDetectors]:
            for ID in each:
                 Current_vehIDs.append(ID)
    return len(Total_VehIDs) - len(Current_vehIDs) - len(Starting_VehIDs)

In [ ]:
result = OB_nbr_VehLeft(60, OBDetectors= OBDetectors)

In [ ]:
TL.getNextSwitch('tl0')

In [ ]:
traci.simulation.getTime()

In [ ]:
for _ in range(15): traci.simulationStep()

In [ ]:
links = TL.getControlledLinks('tl0') 
IBlaneList = [links[i][0][0]for i in range(len(links))]

In [ ]:
[Ln.getWaitingTime(lane) for lane in IBlaneList]

In [ ]:
TL.getNextSwitch(lgt_1)

In [ ]:
state_to_array(TL.getRedYellowGreenState(lgt_1))

In [ ]:
from helper import state_to_array

In [ ]:
laneareas

In [ ]:
for each in laneareas: print(each)

In [ ]:
type(laneareas)

In [23]:
a = np.random.random((10, 8))

In [38]:
b = np.random.random((24,8))
c = [0, 0, 0, 0, 0, 0, 0, 0]

In [39]:
a.shape

(10, 8)

In [40]:
b.shape

(24, 8)

In [41]:
c 

[0, 0, 0, 0, 0, 0, 0, 0]

In [43]:
np.row_stack((a, b, c))

array([[5.40608333e-01, 1.33322844e-01, 4.25301439e-01, 6.32043877e-01,
        1.68328111e-01, 4.46178823e-01, 8.28744387e-01, 6.80077542e-02],
       [6.14629061e-01, 6.07415179e-01, 8.02675079e-01, 1.54467476e-01,
        3.85609940e-01, 3.21468195e-03, 5.69858920e-01, 7.19822679e-01],
       [8.94131110e-02, 8.99352030e-01, 2.55428246e-01, 7.92502856e-01,
        2.67668321e-01, 4.53778042e-01, 5.66488349e-01, 5.84213978e-01],
       [3.87851569e-01, 8.05143551e-01, 9.56181722e-02, 9.26276946e-01,
        2.60333346e-02, 8.39086331e-01, 9.69247872e-01, 1.83022568e-01],
       [5.29808228e-01, 9.95667851e-01, 2.27801743e-01, 9.99501481e-01,
        3.36593506e-01, 5.57150318e-01, 5.58733954e-01, 7.01784205e-01],
       [9.95313825e-02, 4.75424242e-01, 2.33575022e-01, 4.38539576e-01,
        5.21580025e-01, 7.01578199e-01, 2.45866271e-01, 8.78802364e-01],
       [4.15627311e-02, 8.66703387e-01, 9.55184204e-01, 5.41995159e-01,
        2.61963552e-01, 2.10623892e-01, 9.59619467e-01, 3.